In [2]:
import numpy as np
import tensorflow as tf
import time
import os
import sys

import nsm
from nsm import data_utils
from nsm import env_factory
from nsm import graph_factory
from nsm import model_factory
from nsm import agent_factory
from nsm import executor_factory
from nsm import computer_factory
from nsm import word_embeddings

import experiment as exp

FLAGS = tf.app.flags.FLAGS  
tf.app.flags.DEFINE_string('f', '', 'kernel')
# Set the level to tf.logging.INFO if you want to see more information.
tf.logging.set_verbosity(tf.logging.INFO)

Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
# Fill in the path to your data/wikitable folder. 
# By default it is in ~/projects/data/wikitable. 
data_dir= os.path.expanduser('~/Research/SemanticParsing/nsm/data/wikitable')

In [7]:
FLAGS.eval_only = True
FLAGS.eval_use_gpu = False
FLAGS.eval_gpu_id = 0
FLAGS.max_n_mem = 60
FLAGS.eval_file = os.path.join(data_dir, 'processed_input/preprocess_14/data_split_1/dev_split.jsonl')
unittest_file = os.path.join(data_dir, 'processed_input/preprocess_14/data_split_1/train_split_shard_90-0.jsonl')
train_file = os.path.join(data_dir, 'processed_input/preprocess_14/train_examples.jsonl')

# Fill in the output folder and experiment name you want to load.
# By default, load the pretrained model in the repo. 
# FLAGS.output_dir = os.path.expanduser('~/Research/SemanticParsing/nsm/neural-symbolic-machines/table/wtq/')
FLAGS.output_dir = os.path.expanduser('~/Research/SemanticParsing/nsm/data/wikitable/output')
FLAGS.experiment_to_eval = 'debug' # 'pretrained_model'
experiment_config = exp.create_experiment_config()

In [5]:
# Load the agent and the environments in the dev set. 
# This usually takes 15-30 sec. 
# If you want to see the training environments, use train_file instead (takes about 75-150 sec). 
# fns = [FLAGS.eval_file]
FLAGS.table_file = os.path.join(data_dir, 'processed_input/preprocess_14/tables.jsonl')
FLAGS.embedding_file = os.path.join(data_dir, 'raw_input/wikitable_glove_embedding_mat.npy')
FLAGS.vocab_file = os.path.join(data_dir, 'raw_input/wikitable_glove_vocab.json')
FLAGS.en_vocab_file = os.path.join(data_dir, 'processed_input/preprocess_14/en_vocab_min_count_5.json')

fns = [os.path.join(data_dir, 'processed_input/preprocess_14/train_examples.jsonl')]
agent, envs = exp.init_experiment(fns, FLAGS.eval_use_gpu, gpu_id=str(FLAGS.eval_gpu_id))
for env in envs:
    env.punish_extra_work = False
env_dict = dict([(env.name, env) for env in envs])

INFO:tensorflow:14152 examples in dataset.
INFO:tensorflow:2108 tables.
INFO:tensorflow:2045 unique tokens in encoder vocab
INFO:tensorflow:14152 examples in the dataset
INFO:tensorflow:creating environment #0
INFO:tensorflow:creating environment #100
INFO:tensorflow:creating environment #200
INFO:tensorflow:creating environment #300
INFO:tensorflow:Not enough memory slots for example nt-347, which has 75 constants.
INFO:tensorflow:creating environment #400
INFO:tensorflow:creating environment #500
INFO:tensorflow:creating environment #600
INFO:tensorflow:creating environment #700
INFO:tensorflow:creating environment #800
INFO:tensorflow:creating environment #900
INFO:tensorflow:creating environment #1000
INFO:tensorflow:creating environment #1100
INFO:tensorflow:creating environment #1200
INFO:tensorflow:creating environment #1300
INFO:tensorflow:creating environment #1400
INFO:tensorflow:creating environment #1500
INFO:tensorflow:creating environment #1600
INFO:tensorflow:creating en

IOError: [Errno 2] No such file or directory: '/Users/yinpengcheng/Research/SemanticParsing/nsm/data/wikitable/output/debug/graph_config.json'

# Evaluate on 5 environments / questions and show generated programs. 
Use the environment id (for example, nt-34) to find the question and its accompanying table in the website below (from Stanford NLP group). 
https://nlp.stanford.edu/software/sempre/wikitable/viewer/#203-591

In [20]:
# Use 'eval_envs = envs' to evaluate on the whole validation set. Usually takes 8-10 minutes on a laptop. 
eval_envs = envs # envs[5:10]
dev_avg_return, dev_samples, dev_samples_in_beam = exp.beam_search_eval(agent, eval_envs)
print('Accuracy on the selected {} environments are {}'.format(len(eval_envs), dev_avg_return))
print('Show the generated programs:')
print(exp.show_samples(dev_samples, envs[0].de_vocab, env_dict=env_dict))

INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 0: 50 envs
INFO:tensorflow:245 samples in beam, batch 0.
INFO:tensorflow:7.47365903854 sec used in evaluator batch 0.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 1: 50 envs
INFO:tensorflow:248 samples in beam, batch 1.
INFO:tensorflow:7.22429490089 sec used in evaluator batch 1.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 2: 50 envs
INFO:tensorflow:246 samples in beam, batch 2.
INFO:tensorflow:7.89797520638 sec used in evaluator batch 2.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 3: 50 envs
INFO:tensorflow:245 samples in beam, batch 3.
INFO:tensorflow:10.6939949989 sec used in evaluator batch 3.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 4: 50 envs
INFO:tensorflow:246 samples in beam, batch

INFO:tensorflow:eval, batch 37: 50 envs
INFO:tensorflow:241 samples in beam, batch 37.
INFO:tensorflow:7.55130791664 sec used in evaluator batch 37.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 38: 50 envs
INFO:tensorflow:242 samples in beam, batch 38.
INFO:tensorflow:10.3528831005 sec used in evaluator batch 38.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 39: 50 envs
INFO:tensorflow:247 samples in beam, batch 39.
INFO:tensorflow:7.44134783745 sec used in evaluator batch 39.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 40: 50 envs
INFO:tensorflow:244 samples in beam, batch 40.
INFO:tensorflow:7.80551695824 sec used in evaluator batch 40.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 41: 50 envs
INFO:tensorflow:249 samples in beam, batch 41.
INFO:tensorflow:7.82408905029 sec used in evaluat

INFO:tensorflow:Not enough memory slots for example qa_programming, which has 64 constants.
INFO:tensorflow:Not enough memory slots for example qa_programming, which has 64 constants.
INFO:tensorflow:244 samples in beam, batch 74.
INFO:tensorflow:7.33896493912 sec used in evaluator batch 74.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 75: 50 envs
INFO:tensorflow:245 samples in beam, batch 75.
INFO:tensorflow:7.51504993439 sec used in evaluator batch 75.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 76: 50 envs
INFO:tensorflow:248 samples in beam, batch 76.
INFO:tensorflow:10.3487141132 sec used in evaluator batch 76.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 77: 50 envs
INFO:tensorflow:247 samples in beam, batch 77.
INFO:tensorflow:8.05992007256 sec used in evaluator batch 77.
INFO:tensorflow:===========================================

INFO:tensorflow:7.57515597343 sec used in evaluator batch 110.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 111: 50 envs
INFO:tensorflow:243 samples in beam, batch 111.
INFO:tensorflow:7.18578696251 sec used in evaluator batch 111.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 112: 50 envs
INFO:tensorflow:247 samples in beam, batch 112.
INFO:tensorflow:8.32060408592 sec used in evaluator batch 112.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 113: 50 envs
INFO:tensorflow:242 samples in beam, batch 113.
INFO:tensorflow:7.22686195374 sec used in evaluator batch 113.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 114: 50 envs
INFO:tensorflow:246 samples in beam, batch 114.
INFO:tensorflow:6.80771708488 sec used in evaluator batch 114.
INFO:tensorflow:=============================================

INFO:tensorflow:eval, batch 147: 50 envs
INFO:tensorflow:249 samples in beam, batch 147.
INFO:tensorflow:7.56210803986 sec used in evaluator batch 147.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 148: 50 envs
INFO:tensorflow:240 samples in beam, batch 148.
INFO:tensorflow:7.23418903351 sec used in evaluator batch 148.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 149: 50 envs
INFO:tensorflow:248 samples in beam, batch 149.
INFO:tensorflow:10.368131876 sec used in evaluator batch 149.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 150: 50 envs
INFO:tensorflow:247 samples in beam, batch 150.
INFO:tensorflow:7.58992981911 sec used in evaluator batch 150.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 151: 50 envs
INFO:tensorflow:244 samples in beam, batch 151.
INFO:tensorflow:7.40595793724 sec us

INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 184: 50 envs
INFO:tensorflow:243 samples in beam, batch 184.
INFO:tensorflow:8.28483104706 sec used in evaluator batch 184.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 185: 50 envs
INFO:tensorflow:244 samples in beam, batch 185.
INFO:tensorflow:11.3481190205 sec used in evaluator batch 185.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 186: 50 envs
INFO:tensorflow:241 samples in beam, batch 186.
INFO:tensorflow:7.40768194199 sec used in evaluator batch 186.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 187: 50 envs
INFO:tensorflow:244 samples in beam, batch 187.
INFO:tensorflow:8.430341959 sec used in evaluator batch 187.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 188: 50 envs
INFO:tensorflow:24

INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 220: 50 envs
INFO:tensorflow:241 samples in beam, batch 220.
INFO:tensorflow:8.77851200104 sec used in evaluator batch 220.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 221: 50 envs
INFO:tensorflow:248 samples in beam, batch 221.
INFO:tensorflow:8.23786902428 sec used in evaluator batch 221.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 222: 50 envs
INFO:tensorflow:245 samples in beam, batch 222.
INFO:tensorflow:11.5766150951 sec used in evaluator batch 222.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 223: 50 envs
INFO:tensorflow:239 samples in beam, batch 223.
INFO:tensorflow:8.142619133 sec used in evaluator batch 223.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 224: 50 envs
INFO:tensorflow:23

INFO:tensorflow:247 samples in beam, batch 257.
INFO:tensorflow:8.00071597099 sec used in evaluator batch 257.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 258: 50 envs
INFO:tensorflow:241 samples in beam, batch 258.
INFO:tensorflow:7.88566017151 sec used in evaluator batch 258.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 259: 50 envs
INFO:tensorflow:248 samples in beam, batch 259.
INFO:tensorflow:7.44617009163 sec used in evaluator batch 259.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 260: 50 envs
INFO:tensorflow:246 samples in beam, batch 260.
INFO:tensorflow:7.80366110802 sec used in evaluator batch 260.
INFO:tensorflow:==================================================
INFO:tensorflow:eval, batch 261: 50 envs
INFO:tensorflow:248 samples in beam, batch 261.
INFO:tensorflow:11.0343649387 sec used in evaluator batch 261.
INFO:tensorfl

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [38]:
# Save the decoding results for further. 
import json
dev_programs_in_beam_dict = {}
for sample in dev_samples_in_beam:
    name = sample.traj.env_name
    program = agent_factory.traj_to_program(sample.traj, envs[0].de_vocab)
    human_readable_program = exp.to_human_readable_program(program, env_dict[sample.traj.env_name])
    answer = sample.traj.answer
    is_correct = sample.traj.rewards[-1] == 1.
    if name in dev_programs_in_beam_dict:
        # dev_programs_in_beam_dict[name].append((program, answer, sample.prob))
        dev_programs_in_beam_dict[name].append((human_readable_program, answer, sample.prob, is_correct))
    else:
        # dev_programs_in_beam_dict[name] = [(program, answer, sample.prob)]
        dev_programs_in_beam_dict[name] = [(human_readable_program, answer, sample.prob, is_correct)]
        
with open('train_programs.json', 'w') as f:
    json.dump(dev_programs_in_beam_dict, f)

INFO:tensorflow:Not enough memory slots for example qa_programming, which has 75 constants.
INFO:tensorflow:Not enough memory slots for example qa_programming, which has 65 constants.
INFO:tensorflow:Not enough memory slots for example qa_programming, which has 64 constants.
INFO:tensorflow:Not enough memory slots for example qa_programming, which has 75 constants.
INFO:tensorflow:Not enough memory slots for example qa_programming, which has 76 constants.
INFO:tensorflow:Not enough memory slots for example qa_programming, which has 74 constants.
INFO:tensorflow:Not enough memory slots for example qa_programming, which has 74 constants.
INFO:tensorflow:Not enough memory slots for example qa_programming, which has 75 constants.


# Debug the beam search by showing programs in the beam. 

In [1]:
# env_id = 'nt-9585'
# env_id = 'nt-4357'
# env_id = 'nt-7041'

# env_id = 'nt-5557'
# env_id = 'nt-6076'

# env_id = 'nt-3480'

# env_id = 'nt-12815'

# env_id = 'nt-12129' high

# env_id = 'nt-1583'

# env_id = 'nt-8642'

env_id = 'nt-10742'
dev_avg_return, dev_samples, dev_samples_in_beam = exp.beam_search_eval(agent, [env_dict[env_id]])
print('Show the {} programs in beam for environment {}:'.format(len(dev_samples_in_beam), env_id))
print(exp.show_samples(dev_samples_in_beam, envs[0].de_vocab, env_dict=env_dict))

NameError: name 'exp' is not defined

In [235]:
'234-3-1\n\nsadf'.split()

['234-3-1', 'sadf']

In [44]:
envs[0].question_annotation

{u'answer': [u'2004', u'2004.0'],
 u'context': u't_204_590',
 u'entities': [{u'token_end': 16,
   u'token_start': 13,
   u'type': u'string_list',
   u'value': [u'usl a-league']}],
 u'features': [[0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [0],
  [1],
  [0],
  [1],
  [0]],
 u'id': u'nt-0',
 u'in_table': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
 u'pos_tags': [u'WP',
  u'VBD-AUX',
  u'DT',
  u'JJ',
  u'NN',
  u'WRB',
  u'DT',
  u'NN',
  u'VBD-AUX',
  u'DT',
  u'NN',
  u'IN',
  u'DT',
  u'NNP',
  u'DT',
  u'NN',
  u'.'],
 u'processed_tokens': [u'what',
  u'was',
  u'the',
  u'last',
  u'year',
  u'where',
  u'this',
  u'team',
  u'was',
  u'a',
  u'part',
  u'of',
  u'the',
  u'<MISC>',
  u'a',
  u'league',
  u'?'],
 u'prop_features': {u'r.avg_attendance-number': [0],
  u'r.avg_attendance-string': [0],
  u'r.division-number': [0],
  u'r.division-string': [0],
  u'r.league-number': [1],
  u'r.league-string': [1],
  u'r.open_cup-number': [0

In [53]:
def load_nns():
    nn_file = '/Users/yinpengcheng/Research/SemanticParsing/sempre-macro/tables/nn_0/exact_nearest_neighbors.all'
    nns = dict()
    with open(nn_file) as f:
        for line in f:
            q_ids = line.strip().split('\t')
            query_qid = q_ids[0]
            neighbors = q_ids[1].split(',')
            nns[query_qid] = neighbors

    return nns

nns = load_nns()

In [226]:
K = 5
for nn_qid in nns['nt-13279'][:50]:
    print(nn_qid, env_dict[nn_qid].question_annotation['question'])

('nt-2220', u'how many compilation albums were released after 2000?')
('nt-3130', u'how many sites were notified after 2000?')
('nt-10657', u'how many republicans were re-elected after 1920?')
('nt-899', u'how many locomotives were built after 1900?')
('nt-2132', u'how many clubs were runner-ups after the year 2000?')
('nt-2166', u'how many people were appointed after 2011?')
('nt-2978', u'how many albums/singles were released after the year 2000?')
('nt-3124', u'how many conservative party mayors were elected after peder kalve?')
('nt-3315', u'how many ships are named after a state?')
('nt-3469', u'how many were released after power man?')
('nt-3480', u'how many were released after 2004?')
('nt-3860', u'how many candidates were elected in 1990?')
('nt-3897', u'how many abbreviations are after tarmo?')
('nt-4357', u'how many boats were launched after april 1945?')
('nt-4873', u'how many candidates were elected in 1971?')
('nt-5557', u'how many festivals were held after 2012?')
('nt-607

In [147]:
import editdistance

def get_canonical_program_signiture(program):
    stack = []
    i = 0
    while i < len(program):
        token = program[i]
        if token == '(':
            new_expr = []
            stack.append(new_expr)
        elif token == ')':
            pass
        elif token == '<END>':
            pass
        else:
            stack[-1].append(token)
            
        i += 1
            
    program_sig = [expr[0] for expr in stack]
    
    return program_sig


def compute_program_similarity(program1, program2):
    program_sig1 = get_canonical_program_signiture(program1)
    program_sig2 = get_canonical_program_signiture(program2)
    
    # print(program_sig1, program_sig2)
    sim = editdistance.eval(program_sig1, program_sig2)
    return sim

In [211]:
q_id = 'nt-1422'
K = 5

all_predictions_dict = dev_programs_in_beam_dict

def compute_consistency_score(q_id, given_cand_program=None, K=5, debug=False):
    if debug:
        print('Question: ' + env_dict[q_id].question_annotation['question'])
    
    similiar_questions = nns[q_id][:K]
    similiar_questions = [q for q in similiar_questions if q in all_predictions_dict]
    cand_programs = list(all_predictions_dict[q_id])
    if given_cand_program is not None:
        cand_programs = [(given_cand_program, None, None, None)] + cand_programs
    
    supports = []
    for i, (cand_program, answer, prob, is_correct) in enumerate(cand_programs):
        support = 0.
        for nn_qid in similiar_questions:
            similar_question = env_dict[nn_qid].question_annotation['question']
            if debug:
                print(' Similiar Question({}): {}'.format(nn_qid, similar_question))

            nn_question_predictions = all_predictions_dict[nn_qid]
            for nn_cand_program, _, nn_program_prob, nn_program_is_correct in nn_question_predictions:
                if nn_program_is_correct:
                    dist = compute_program_similarity(cand_program, nn_cand_program)
                    support += -dist # * nn_program_prob

                    if debug:
                        print('  Similiar Program: {} ||| dist={} ||| is_correct={}'.format(' '.join([str(x) for x in nn_cand_program]), dist, nn_program_is_correct))
        
        if debug:
            print('')
            print('Prediction: ' + ' '.join([str(x) for x in cand_program]) + ' ||| Score={}, is_correct={}'.format(support, is_correct))
            print('\n')
        
        supports.append(support)
    
    return supports

compute_consistency_score('nt-2879', given_cand_program='( filter_greater all_rows r.voter_turnout-number ) ( count v0 ) <END>'.split(' '), debug=True)


Question: how many parties had more than 50 seats in the third duma?
 Similiar Question(nt-1422): how many had voter turnouts more than 50%?
  Similiar Program: ( minimum all_rows r.voter_turnout-number ) ( filter_greater all_rows v0 r.voter_turnout-number ) ( count v1 ) <END> ||| dist=1 ||| is_correct=True
 Similiar Question(nt-5534): how many rounds had more than 20,000 spectators in attendance?
  Similiar Program: ( filter_ge all_rows 20000 r.attendance-number ) ( count v0 ) <END> ||| dist=1 ||| is_correct=True
  Similiar Program: ( filter_greater all_rows 20000 r.attendance-number ) ( count v0 ) <END> ||| dist=0 ||| is_correct=True
 Similiar Question(nt-5992): how many districts have more than 100,000 people in this city?
  Similiar Program: ( filter_ge all_rows 100000 r.population-number ) ( count v0 ) <END> ||| dist=1 ||| is_correct=True
  Similiar Program: ( filter_greater all_rows 100000 r.population-number ) ( count v0 ) <END> ||| dist=0 ||| is_correct=True
 Similiar Question(

[-5.0, -6.0, -10.0, -10.0, -10.0, -19.0]

In [205]:
# compute support score for all examples
consistency_scores_dict = {q_id: compute_consistency_score(q_id, K=3) for q_id in all_predictions_dict}

In [206]:
print('Original Acc.', np.average([programs[0][3] for programs in dev_programs_in_beam_dict.values()]))

def _score_func(program, q_id, pos, w=1.):
    return program[2] + w * consistency_scores_dict[q_id][pos]

def rerank(w):
    results = []
    for q_id, programs in dev_programs_in_beam_dict.items():
        best_program_idx = np.argmax([_score_func(proc, q_id, i, w) for i, proc in enumerate(programs)])
        best_program = programs[best_program_idx]
        
        results.append(best_program[3])
        
    return np.average(results)

for w in np.arange(0., 1.0, 0.01):
    print('w={}, Rerank Acc.={}'.format(w, rerank(w)))

('Original Acc.', 0.5232673267326733)
w=0.0, Rerank Acc.=0.523267326733
w=0.01, Rerank Acc.=0.523267326733
w=0.02, Rerank Acc.=0.523338048091
w=0.03, Rerank Acc.=0.523833097595
w=0.04, Rerank Acc.=0.523974540311
w=0.05, Rerank Acc.=0.523338048091
w=0.06, Rerank Acc.=0.523338048091
w=0.07, Rerank Acc.=0.523055162659
w=0.08, Rerank Acc.=0.522418670438
w=0.09, Rerank Acc.=0.522560113154
w=0.1, Rerank Acc.=0.521923620934
w=0.11, Rerank Acc.=0.521004243281
w=0.12, Rerank Acc.=0.51973125884
w=0.13, Rerank Acc.=0.518175388967
w=0.14, Rerank Acc.=0.517397454031
w=0.15, Rerank Acc.=0.516690240453
w=0.16, Rerank Acc.=0.515983026874
w=0.17, Rerank Acc.=0.51414427157
w=0.18, Rerank Acc.=0.513083451202
w=0.19, Rerank Acc.=0.512446958982
w=0.2, Rerank Acc.=0.510042432815
w=0.21, Rerank Acc.=0.509476661952
w=0.22, Rerank Acc.=0.508628005658
w=0.23, Rerank Acc.=0.508062234795
w=0.24, Rerank Acc.=0.507072135785
w=0.25, Rerank Acc.=0.502687411598
w=0.26, Rerank Acc.=0.50198019802
w=0.27, Rerank Acc.=0.5

In [238]:
import json

new_programs = dict()
for i in range(0, 30):
    file_path = '/home/pengchey/projects/data/wikitable/output/random_explore/program_shard_{}-1000.json'.format(i)
    
    shard_programs = json.load(open(file_path))
    new_programs.update(shard_programs)

IOError: [Errno 2] No such file or directory: '/home/pengchey/projects/data/wikitable/output/random_explore/program_shard_0-1000.json'